## How to Grid Search Hyperparameters for Deep Learning Models in Python with Keras

- https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

In [1]:
# Use scikit-learn to grid search the learning rate and momentum
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import SGD
from scikeras.wrappers import KerasClassifier

In [2]:
# Function to create model, required for KerasClassifier
def create_model():
    # create model
    model = Sequential()
    model.add(Dense(12, input_shape=(8,), activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    
    # return values
    return model

In [3]:
# fix random seed for reproducibility
tf.random.set_seed(seed=1234)

In [4]:
# load dataset
dataset = np.loadtxt("dataset/diabetes.csv", delimiter=",")

In [5]:
# split into input (X) and output (Y) variables
X = dataset[:,0:8]
Y = dataset[:,8]

In [6]:
# show to dataset
pd.concat([
    pd.DataFrame(X, columns=['x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8']),
    pd.DataFrame(Y, columns=['Y'])
], axis=1)

,x1,x2,x3,x4,x5,x6,x7,x8,Y
0,6.0,148.0,72.0,35.0,0.0,33.6,0.627,50.0,1.0
1,1.0,85.0,66.0,29.0,0.0,26.6,0.351,31.0,0.0
2,8.0,183.0,64.0,0.0,0.0,23.3,0.672,32.0,1.0
3,1.0,89.0,66.0,23.0,94.0,28.1,0.167,21.0,0.0
4,0.0,137.0,40.0,35.0,168.0,43.1,2.288,33.0,1.0
...,...,...,...,...,...,...,...,...,...
763,10.0,101.0,76.0,48.0,180.0,32.9,0.171,63.0,0.0
764,2.0,122.0,70.0,27.0,0.0,36.8,0.340,27.0,0.0
765,5.0,121.0,72.0,23.0,112.0,26.2,0.245,30.0,0.0
766,1.0,126.0,60.0,0.0,0.0,30.1,0.349,47.0,1.0


In [7]:
# create model
model = KerasClassifier(model=create_model, loss="binary_crossentropy", optimizer="SGD", epochs=100, batch_size=10, verbose=0)

In [8]:
# define the grid search parameters
learn_rate = [0.001, 0.01, 0.1, 0.2, 0.3]
momentum = [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]

In [9]:
param_grid = dict(optimizer__learning_rate=learn_rate, optimizer__momentum=momentum)

In [10]:
param_grid

{'optimizer__learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3],
 'optimizer__momentum': [0.0, 0.2, 0.4, 0.6, 0.8, 0.9]}

In [11]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)

In [12]:
grid_result = grid.fit(X, Y)

In [14]:
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Best: 0.687500 using {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}


In [15]:
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

0.687500 (0.019137) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.0}
0.669271 (0.009207) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.2}
0.645833 (0.018688) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.4}
0.684896 (0.008027) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.6}
0.671875 (0.008438) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.8}
0.652344 (0.003189) with: {'optimizer__learning_rate': 0.001, 'optimizer__momentum': 0.9}
0.665365 (0.013279) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.0}
0.662760 (0.012075) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.2}
0.671875 (0.028348) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.4}
0.648438 (0.005524) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.6}
0.651042 (0.001841) with: {'optimizer__learning_rate': 0.01, 'optimizer__momentum': 0.8}
0.651042 (0.001